In [ ]:
import numpy as np
import os 
# Plotoimg Functions
import matplotlib
import time 
import scipy
import matplotlib.pyplot as mplt 

import pylab as PLT
import plotly
import plotly.tools as tls
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from sklearn.neighbors import KNeighborsClassifier
from pprint import pprint
from sklearn.metrics.pairwise import euclidean_distances
# import ground truth implementations for comparison
plotly.offline.init_notebook_mode()
data_dir = './'

In [ ]:
# Create the distance graph for all the input 
from itertools import product
def get_nearest_neighbors_edges(x, k=6, distance_metric = 'euclidean'):
    n_dim, n_samples = x.shape
    edges = [] 
    pairwise_distances = scipy.spatial.distance.squareform(
                         scipy.spatial.distance.pdist(x.T, metric=distance_metric))
    for s_id in np.arange(n_samples):
        neighbors = np.argsort(pairwise_distances[s_id, :])
        closest_neighbors = neighbors[:k+1]
        for neigh_id in closest_neighbors:
            edges.append([s_id, neigh_id, pairwise_distances[s_id, neigh_id]])
    return edges


def floyd_warshall(x, k=6, distance_metric='euclidean'):
    edges = get_nearest_neighbors_edges(x, k=6)
    n_dim, n_samples = x.shape
    D = np.full((n_samples, n_samples), np.finfo(np.float16).max, dtype=np.float32)
    for (st_ind, end_id, distance) in edges:
        D[st_ind][end_id] = distance
    for k in range(n_samples):
        D = np.minimum(D, D[:,int(k),np.newaxis] + D[np.newaxis,int(k),:]) 
    for i in range(n_samples):
        for j in range(n_samples):
            D[i][j] = min(D[i][j], D[j][i])
    return D
        

In [ ]:
# Load the digtis from the given file
digits_path = os.path.join(data_dir, 'digits-labels.npz')
load_dic = np.load(digits_path)
digits_labels = load_dic['l']
digits_data = load_dic['d']

n_total_digits = digits_data.shape[1]
# just check one input datapoint and its corresponding label: 
indx = np.random.randint(n_total_digits)
print(n_total_digits)
print("Showing a digit: {}".format(digits_labels[indx]))
mplt.imshow(digits_data[:, indx].T.reshape(28, 28).T)

In [ ]:
def pairwise_euclidean(X):
    return euclidean_distances(X, X)

def poll_all_directions(direction_vecs,
                        direction_inds): 
    return direction_vecs 

def get_steepest_direction(directions,
                           i,
                           Z,
                           r_k,
                           D_z,
                           D_target,
                           D_diff,
                           error_k,
                           new_errors,
                           new_diffs,
                           new_distances,
                           new_z,
                           history_directions):
    
    for j in np.arange(new_distances.shape[1]):
        new_z[:, j] = Z[i, :] + r_k * directions[j]
            
        new_distances[:, j] = np.sqrt(np.sum((Z - new_z[:, j]) ** 2, axis = 1))
        new_distances[i, j] = 0.
        
        new_diffs[:, j] = np.abs(D_target[:, i] - new_distances[:, j])
        new_sum_diff = 2 * np.sum(new_diffs[:, j])
        new_errors[j] = error_k - 2 * np.sum(D_diff[i, :]) + new_sum_diff 
        
        if new_errors[j] < 0:
            print("Psola")
            print(new_diff)
            print(2 * np.sum(D_diff[i, :]))
    
    steepest = np.argmin(new_errors)
    history_directions[i][steepest] += 1
    
    return (new_z, steepest, new_errors, 
            new_diffs, new_distances)  
    

def coordinate_search_MDS(D_target, 
                          dim_target, 
                          epochs=1000,
                          radius_thresh = 10e-4, 
                          conv_thresh=10e-2, 
                          r=20):
    n_points = D_target.shape[0]
    Z = np.random.uniform(low=0.0, high=1.0, size=(n_points, dim_target))
#     Z = np.array([[0, 0], [1,1], [4,4]])
    D_z = pairwise_euclidean(Z)
    
    
    
    D_diff = np.abs(D_z - D_target) 
    error = np.zeros(epochs)
    error[0] = np.sum(D_diff)
    rs = np.zeros(epochs)
    rs[1] = r
    n_directions = 2 * dim_target
    new_errors = np.zeros(n_directions, dtype=np.float32)
    new_diffs = np.zeros((n_points, n_directions), dtype=np.float32)
    new_distances = np.zeros((n_points, n_directions), dtype=np.float32)
    new_z = np.zeros((dim_target, n_directions), dtype=np.float32)

    direction_vecs = np.concatenate((np.eye(dim_target), 
                                    - np.eye(dim_target)), axis=0)
    direction_inds = np.arange(n_directions)
    
    history_directions = np.zeros((n_points, n_directions))
    
    for k in np.arange(1, epochs):
        error[k] = error[k-1]
        
        evaltime = 0
        updatetime = 0
        
        for i in np.arange(n_points):
            possible_directions = poll_all_directions(direction_vecs,
                                                      direction_inds)
            
            before = time.time()
            (Z_i_new,
             steepest_direction,
             new_errors,
             new_diff_for_i,
             new_distances_for_i) = get_steepest_direction(possible_directions,
                                                           i,
                                                           Z, 
                                                           rs[k],
                                                           D_z,
                                                           D_target,
                                                           D_diff,
                                                           error[k],
                                                           new_errors,
                                                           new_diffs,
                                                           new_distances,
                                                           new_z,
                                                           history_directions)
            evaltime += time.time() - before
            
            before = time.time()
            if new_errors[steepest_direction] < error[k]:
            # Successful iteration
                Z[i] = Z_i_new[:, steepest_direction]
                error[k] = new_errors[steepest_direction]
                D_z[i, :] = new_distances_for_i[:, steepest_direction].T
                D_z[:, i] = new_distances_for_i[:, steepest_direction]
                D_diff[i, :] = new_diff_for_i[:, steepest_direction].T
                D_diff[:, i] = new_diff_for_i[:, steepest_direction]
            updatetime += time.time() - before
            
        if (error[k-1] - error[k]) / (error[k-1] - 1e-8)  < conv_thresh:
            rs[k+1] = rs[k] / 2.
        else:
            rs[k+1] = rs[k]
            
        if rs[k] < radius_thresh:
            break
            
        print(error[k], rs[k])
        print(np.sum(np.abs(D_target - D_z)))
        print(evaltime, updatetime)
        print(np.mean(np.max(history_directions, axis =1)))
    return Z, D_z, error


In [ ]:
selected_digits = [0, 1, 3, 9]
groups_counts = np.bincount(digits_labels)
sorted_numberes_indxes = digits_labels.argsort()
sums_of_counts = np.cumsum(groups_counts)
list_of_sel_digits_data = [sorted_numberes_indxes[sums_of_counts[dig-1] % sums_of_counts[-1]:sums_of_counts[dig]]
                           for dig in selected_digits]
selected_digits_data = digits_data[:, np.concatenate(list_of_sel_digits_data, axis=0)]
selected_digits_labels = digits_labels[np.concatenate(list_of_sel_digits_data, axis=0)]

n_digits = 1000
selected_indexes = np.random.choice(selected_digits_labels.shape[0], 
                                    n_digits, replace=False)
sel_digits_labels = selected_digits_labels[selected_indexes]
sel_digits_data = selected_digits_data[:, selected_indexes]

D_target = pairwise_euclidean(sel_digits_data.T)
# D_target = floyd_warshall(sel_digits_data.T, k=6, 
#                             distance_metric='euclidean')

In [ ]:
before = time.time()
Z, D_z, errors = coordinate_search_MDS(D_target, 2, 
                                       conv_thresh=5*10e-4,
                                       r=40)
print(time.time() - before)

In [ ]:
from sklearn.manifold import MDS as classic_MDS
# before = time.time()
# embedding = classic_MDS(n_components=2, n_init=1, n_jobs=1)
# X_transformed = embedding.fit_transform(sel_digits_data.T)
# print(time.time() - before)

before = time.time()
embedding = classic_MDS(n_components=2, n_init=1, n_jobs=1, dissimilarity='precomputed')
X_transformed = embedding.fit_transform(D_target)
print(time.time() - before)
print(embedding.stress_ / 2.)

In [ ]:
import matplotlib.pyplot as PLT
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

def digits_scatter2D(embedding, digit_labels, digit_true_data, title=''):
    color_mapper = ['green' ,'blue', 'sienna', 'orange', 'purple',  'grey', 'aqua', 'pink', 'sienna', 'red']
    all_inds = np.arange(embedding.shape[0])
    sel_inds = np.random.choice(all_inds, size=200, replace=False)

    fig = PLT.gcf()
    fig.set_size_inches(18.5, 18.5)
    fig.clf()
    ax = PLT.subplot(111)
    
    xys = [embedding[ind, :] for ind in sel_inds]
    xs = [xy[0] for xy in xys]
    ys = [xy[1] for xy in xys]
    PLT.scatter(xs, ys, s=10000,
               c=[color_mapper[digit_labels[ind]] for ind in sel_inds],
               alpha=0.5)

    for ind in sel_inds:
        xy = embedding[ind, :]
        img_to_show = digit_true_data[:, ind].T.reshape(28,28).T

        # add a first image
        imagebox = OffsetImage(img_to_show, zoom=1.1)
        
        ab = AnnotationBbox(imagebox, xy,
            xybox=(-5., 5.),
            xycoords='data',
            boxcoords="offset points",
            arrowprops=dict(arrowstyle="->"))                                  
        ax.add_artist(ab)


    # rest is just standard matplotlib boilerplate
    ax.grid(True)
    fig.suptitle(title, fontsize=20)
    PLT.draw()
    PLT.show()

In [ ]:
digits_scatter2D(Z, sel_digits_labels, sel_digits_data, title='Coordinate Search MDS')
digits_scatter2D(X_transformed, sel_digits_labels, sel_digits_data, title='SMACOF MDS')

In [ ]:
import sklearn.datasets
swiss_xyz, swiss_t = sklearn.datasets.make_swiss_roll(n_samples=1000, 
                                                      noise=.0, 
                                                      random_state=None)
def matplotlib_to_plotly(cmap, pl_entries):
    h = 1.0/(pl_entries-1)
    pl_colorscale = []
    
    for k in range(pl_entries):
        C = map(np.uint8, np.array(cmap(k*h)[:3])*255)
        print(list(C))
        pl_colorscale.append([k*h, 'rgb'+str((C[0], C[1], C[2]))])
        
    return pl_colorscale

def plot_initial_swissroll(X, X_t, title='Swissroll 3D'):
    p1 = go.Scatter3d(x=X[:, 0], y=X[:, 1], z=X[:, 2],
                          mode='markers', 
                          marker=dict(color=X_t, 
                                      colorscale='Viridis',
                                      size=4,
                                      showscale=False,
                                      line=dict(color='black', width=1)))
    layout = go.Layout(title = title)
    fig = dict(data=[p1], layout=layout)
    plotly.offline.iplot(fig, filename=title)

plot_initial_swissroll(swiss_xyz, swiss_t, title='Swissroll 3D')
# cmap = matplotlib_to_plotly(mplt.cm.Spectral, 3)

In [ ]:
Geodesic_D = floyd_warshall(swiss_xyz.T, k=6, 
                            distance_metric='euclidean')

In [ ]:
def plot_embedded_swissroll(X, X_t, title='Swissroll 2D'):
    p1 = go.Scatter(x=X[:, 0], y=X[:, 1],
                          mode='markers', 
                          marker=dict(color=X_t, 
                                      colorscale='Viridis',
                                      showscale=False,
                                      line=dict(color='black', width=1)))
    layout = go.Layout(title = title)
    fig = dict(data=[p1], layout=layout)
    plotly.offline.iplot(fig, filename=title)

In [ ]:
before = time.time()
embedding = classic_MDS(n_components=2, n_init=1, 
                        n_jobs=1, dissimilarity='precomputed',
                        verbose=5)
X_transformed = embedding.fit_transform(Geodesic_D)
print(time.time() - before)    

In [ ]:
before = time.time()
Z, D_z, errors = coordinate_search_MDS(Geodesic_D, 2, conv_thresh=2*10e-4,
                                       r=1)
print(time.time() - before)

In [ ]:
plot_embedded_swissroll(Z, swiss_t, title='Pattern Search MDS Swissroll 2D')
plot_embedded_swissroll(X_transformed, swiss_t, title='Classic MDS Swissroll 2D')